In [196]:
import numpy as np
import pandas as pd
import heapq
import random

# Generation of initial population of random antibodies

In [197]:
# m: number of machines
# p: number of parts
# population_size : number of antibody of the initial population
# seed: seed for reproducible results
def generation_initial_population(p, m, population_size):
    MaxCell = min(p,m) #calculation of max number of cells
    number_of_zeros = MaxCell - 1 #number of zeros in each antibody
    antibodies = np.empty(shape=(population_size, p+m+number_of_zeros), dtype=int)
    antibody = np.append(np.array([*range(1,p+m+1)]), np.zeros(number_of_zeros,dtype=int))
    for i in range(0,population_size):
        # np.random.seed(seed) 
        np.random.shuffle(antibody) #random positions in the array
        antibodies[i] = antibody
    return antibodies

In [198]:
antibodies = generation_initial_population(p = 7, m = 5, population_size = 3)
antibodies

array([[ 5,  0,  1,  3,  7,  0, 11,  4,  0, 10,  9,  0,  8,  6,  2, 12],
       [ 0, 10, 12,  5,  0,  3,  9,  1,  0, 11,  4,  0,  8,  6,  2,  7],
       [ 7,  5, 10,  0, 12,  0,  0,  8,  9,  6, 11,  4,  0,  1,  3,  2]])

# Evaluate all existing antibodies and compute their affinities

Lectura de los datos del problema + Traducción del formato del dataset a matriz trabajo-estación

In [199]:
def part_machine_incidence_matrix(data):    
    f = open(data,'r')
    lines = [line.split('\n') for line in f]
    f.close()

    m,p = [int(num) for num in lines[8][0].split(' ')] #m: number of machines, p: number of parts

    machines=[[m,p]]
    for i in range(9,9+m):
        machines.append([int(lines[i][0].split(' ')[j]) for j in range(1,m)])

    columns, rows = ['M'+str(i) for i in range(1,m+1)], ['P'+str(i) for i in range(1,p+1)]
    m_p_matrix = pd.DataFrame(columns= columns, index= rows)

    ones_zeros = []
    number_of_operations = 0 # number of 'ones' in the part-machine matrix
    for i in range(1,len(machines)):
        aux = []
        for j in range(1,p+1):
            if j in machines[i]: 
                aux.append(1)
                number_of_operations = number_of_operations +1
            else: aux.append(0)
        ones_zeros.append(aux)

    for i in range(0,len(columns)):
        m_p_matrix[columns[i]] = ones_zeros[i]

    return m_p_matrix, m, p, number_of_operations, columns, rows

In [200]:
matrix, m, p, number_of_operations, columns, rows = part_machine_incidence_matrix('data/instances/testset_a/5x7_Waghodekar_Sahu(1984)[Problem-2].txt')
matrix

,M1,M2,M3,M4,M5
P1,1,0,0,1,0
P2,0,1,0,1,1
P3,0,1,1,1,0
P4,0,1,1,1,1
P5,1,1,1,0,1
P6,1,0,1,0,1
P7,1,0,0,0,0


## Decodificación de anticuerpos
1er Paso: Separación de celdas.

In [201]:
def cell_identification(antibodies):
    total_cells = []
    for antibodie in antibodies:
        # print("antibodie", antibodie)
        flag = 1 #bandera que indica si el num anterior es un cero
        cells, cell = [],[]
        i = 0
        for num in antibodie:
            if (num == 0): 
                if flag == 0: 
                    i=i+1
                    cells.append(cell)
                    # print(cell)
                    cell = []
                flag = 1
            else: 
                cell.append(num)
                # print(num)
                flag = 0
                if num == antibodie[len(antibodie)-1]:
                    cells.append(cell)
        total_cells.append(cells)
        # print("cells", cells)
    # print("list of all cells for all antibodies",total_cells)
    # print("cells for antibody 1",total_cells[0])
    return total_cells

In [202]:
total_cells = cell_identification(antibodies = antibodies)
total_cells

[[[5], [1, 3, 7], [11, 4], [10, 9], [8, 6, 2, 12]],
 [[10, 12, 5], [3, 9, 1], [11, 4], [8, 6, 2, 7]],
 [[7, 5, 10], [12], [8, 9, 6, 11, 4], [1, 3, 2]]]

reorganizar filas y columnas de la matriz maquinas-trabajos en función de lo descrito por el anticuerpo

In [203]:
def decode_cells(total_cells):
    total_machines, total_parts = [], []
    for antibodie in total_cells:
        # print("antibodie",antibodie)
        machines = []
        parts = []
        decoded_antibodie = antibodie
        for i in range(0,len(antibodie)):
            # print(antibodie[i])
            for j in range(0,len(antibodie[i])):
                if antibodie[i][j] <= p: 
                    parts.append(rows[antibodie[i][j]-1])
                    decoded_antibodie[i][j] = rows[antibodie[i][j]-1]
                else: 
                    machines.append(columns[antibodie[i][j]-p-1])
                    decoded_antibodie[i][j] = columns[antibodie[i][j]-p-1]
            antibodie = decoded_antibodie
        total_machines.append(machines)
        total_parts.append(parts)
        # print("decoded antibodies",antibodie)

    return total_machines, total_parts
    # print(total_machines)
    # print(total_parts)

In [204]:
total_machines, total_parts = decode_cells(total_cells=total_cells)

decoded antibodies [['P5'], ['P1', 'P3', 'P7'], ['M4', 'P4'], ['M3', 'M2'], ['M1', 'P6', 'P2', 'M5']]
decoded antibodies [['M3', 'M5', 'P5'], ['P3', 'M2', 'P1'], ['M4', 'P4'], ['M1', 'P6', 'P2', 'P7']]
decoded antibodies [['P7', 'P5', 'M3'], ['M5'], ['M1', 'M2', 'P6', 'M4', 'P4'], ['P1', 'P3', 'P2']]


Representación de la matriz: usamos total_machines y total_parts, donde hemos colocado en orden las máquinas y los trabajos respectivamente.


In [205]:
def create_machine_part_matrix(matrix, antibodies, total_machines, total_parts):
    antibody_matrices = []
    for i in range(0,len(antibodies)):
        antibodie_matrix = matrix.loc[:,total_machines[i]]
        antibodie_matrix = antibodie_matrix.loc[total_parts[i]]
        antibody_matrices.append(antibodie_matrix)
    return antibody_matrices


antibody_matrices = create_machine_part_matrix(matrix=matrix,
                                                antibodies=antibodies, 
                                                total_machines=total_machines, 
                                                total_parts=total_parts)

In [206]:
def evaluate_antibodies(antibody_matrices, total_cells, number_of_operations): #m and p should be added as parameters
    exceptions, voids, penalties = [],[],[]
    for i in range(0,len(total_cells)):
        # print("\n",total_cells[i])
        # print(antibody_matrices[i])
        void, exception, penalty, operations_number = 0,0,0,0 #operations_number is number of 'ones' in part-machine matrix
        cell_machine_flag, cell_flag_part = 0,0
        for cell in total_cells[i]:
            # print(cell)
            machines, parts = [], []
            machine_flag, part_flag = 0,0 #flag variables for calculation of penalties
            for mp in cell:
                if mp[0] == 'M': 
                    machines.append(mp)
                    machine_flag = 1
                    # print(machines)
                if mp[0] == 'P': 
                    parts.append(mp)
                    part_flag = 1
                    # print(parts)
            if machine_flag == 0: cell_machine_flag = 1
            if part_flag == 0: cell_flag_part = 1
            # print(i, cell_machine_flag, cell_flag_part)
            for machine in machines:
                for part in antibody_matrices[i].index:
                    if part in parts and antibody_matrices[i][machine][part] == 0: 
                        void = void+1
                        # print("void",machine, part)
                    if part not in parts and antibody_matrices[i][machine][part] == 1:
                        exception = exception+1
                        # print("exception",machine, part)
            # print(void, exception)
            # print("\n")
        voids.append(void)
        exceptions.append(exception)
        penalty = 0.5 * (cell_machine_flag + cell_flag_part)
        # print(penalty)
        penalties.append(penalty)
    # print("voids",voids)
    # print("exceptions",exceptions)
    efficacies, affinities = [], []
    # matrix_dimension = m*p
    for i in range(0,len(total_cells)):
        exceptions_ratio = exceptions[i]/number_of_operations
        voids_ratio = voids[i]/number_of_operations
        efficacy= (1-exceptions_ratio)/(1+voids_ratio)
        efficacies.append(efficacy)
        affinities.append(efficacy - efficacy * penalties[i])
    return efficacies, affinities, voids, exceptions


In [207]:
efficacies, affinities, voids, exceptions = evaluate_antibodies(antibody_matrices = antibody_matrices, 
                                                                total_cells=total_cells,
                                                                number_of_operations=number_of_operations)
print(efficacies)
print(affinities)

[0.19047619047619044, 0.27272727272727276, 0.16666666666666663]
[0.0, 0.27272727272727276, 0.0]


# Select N% of antibodies with highest affinities & Clone selected antibodies

con el parámetro p de probabilities de np.random.choice podemos pasar un vector de probabilidades para el sampleo

In [208]:
def antibodies_selection(antibodies, N, affinities):
    population_pool = len(antibodies)
    sel_probabilities = affinities/np.sum(affinities) #selection probabilities
    size = round(population_pool*N)
    nonzero_affinities = np.count_nonzero(sel_probabilities != 0)

    if nonzero_affinities < size: # if there are more antibodies to be selected than affinities different from zero
        print("MORE antibodies to be selected than affinities different from zero")
        print(sel_probabilities)
        print("number of antibodies to be selected",size)
        positions_antibodies_selected = np.random.choice(population_pool, size=nonzero_affinities, replace=False, p = sel_probabilities)
        sel_probabilities[:] = 1
        sel_probabilities[positions_antibodies_selected] = 0
        sel_probabilities = sel_probabilities/np.sum(sel_probabilities)
        print("new probabilities",sel_probabilities)
        add_number_of_antibodies = size - nonzero_affinities
        add_positions_antibodies_selected = np.random.choice(population_pool, size=add_number_of_antibodies, replace=False, p = sel_probabilities)
        positions_antibodies_selected = np.concatenate((positions_antibodies_selected, add_positions_antibodies_selected), axis = 0)
        antibodies_selected = antibodies[positions_antibodies_selected.tolist()]
        # return print("¡¡ERROR!! en la selección inicial de anticuerpos. INSUFICIENTES ANTICUERPOS CON AFINIDAD > 1")
        return antibodies_selected, positions_antibodies_selected
    else:
        # print("LESS antibodies to be selected than affinities different from zero")
        # print(sel_probabilities)
        positions_antibodies_selected = np.random.choice(population_pool, size=size, replace=False, p = sel_probabilities)
        antibodies_selected = antibodies[positions_antibodies_selected.tolist()]
        return antibodies_selected, positions_antibodies_selected

N = np.random.rand(1)
print("N% =",N[0])
cloned_antibodies, positions_antibodies_selected = antibodies_selection(antibodies=antibodies, N=N[0], affinities= affinities)

print("positions of selected antibodies",positions_antibodies_selected)
cloned_antibodies

N% = 0.24440104110442162
positions of selected antibodies [1]


array([[ 0, 10, 12,  5,  0,  3,  9,  1,  0, 11,  4,  0,  8,  6,  2,  7]])

# Mutation operator

Aqui estoy comentiendo un fallo y es que realizo la mutación en todos los casos.
Se debe comparar la efficacy del anticuerpo antes y después de la mutación y quedarse con el mejor.

## Maturate cloned antibodies

In [209]:
def mutate_cloned_antibodies(cloned_antibodies):
    for antibodie in cloned_antibodies:
        # print(antibodie)
        positions = np.random.choice(len(antibodie),size=2,replace=False)
        # print(positions)
        antibodie[positions[0]], antibodie[positions[1]] = antibodie[positions[1]], antibodie[positions[0]]
    return cloned_antibodies

In [210]:
mutate_cloned_antibodies(cloned_antibodies = cloned_antibodies)

array([[ 0, 10, 12,  5,  0,  3,  9,  1, 11,  0,  4,  0,  8,  6,  2,  7]])

## Evaluate cloned antibodies

In [211]:
cloned_total_cells = cell_identification(cloned_antibodies)
cloned_total_cells

[[[10, 12, 5], [3, 9, 1, 11], [4], [8, 6, 2, 7]]]

In [212]:
cloned_total_machines, cloned_total_parts = decode_cells(cloned_total_cells)

decoded antibodies [['M3', 'M5', 'P5'], ['P3', 'M2', 'P1', 'M4'], ['P4'], ['M1', 'P6', 'P2', 'P7']]


In [213]:
cloned_antibody_matrices = create_machine_part_matrix(matrix = matrix,
                                                        antibodies=cloned_antibodies, 
                                                        total_machines = cloned_total_machines, 
                                                        total_parts= cloned_total_parts)

In [214]:
cloned_efficacies, cloned_affinities, cloned_voids, cloned_exceptions = evaluate_antibodies(cloned_antibody_matrices,
                                                                                           cloned_total_cells,
                                                                                           number_of_operations)
print("Pool\n",antibodies)
print(efficacies)
print(affinities)

print("\nCloned\n",cloned_antibodies)
print(cloned_efficacies)
print(cloned_affinities)

Pool
 [[ 5  0  1  3  7  0 11  4  0 10  9  0  8  6  2 12]
 [ 0 10 12  5  0  3  9  1  0 11  4  0  8  6  2  7]
 [ 7  5 10  0 12  0  0  8  9  6 11  4  0  1  3  2]]
[0.19047619047619044, 0.27272727272727276, 0.16666666666666663]
[0.0, 0.27272727272727276, 0.0]

Cloned
 [[ 0 10 12  5  0  3  9  1 11  0  4  0  8  6  2  7]]
[0.3181818181818181]
[0.15909090909090906]


## Add R% of best cloned antibodies to the pool of antibodies

select the R% of best cloned antibodies and adds them to the pool of antibodies

In [215]:
def select_best_cloned_antibodies(antibodies, cloned_antibodies, efficacies, 
                                    cloned_efficacies,
                                    affinities,
                                    cloned_affinities, 
                                    R, 
                                    positions_antibodies_selected):
    #if antibody efficacy improves after mutation, we keep the mutated antibody, otherwise we dismiss it.
    j = 0
    for i in range(0,len(cloned_antibodies)):
        if cloned_efficacies[i] < efficacies[(positions_antibodies_selected[i])]:
            print("clon {} presenta peor eficacia al ser mutado".format(cloned_antibodies[i]))
            print("eficacia clon mutado",cloned_efficacies[i])
            print("eficacia clon sin mutar",efficacies[(positions_antibodies_selected[i])])
            cloned_antibodies[i] = antibodies[(positions_antibodies_selected[i])]
            cloned_efficacies[i] = efficacies[(positions_antibodies_selected[i])]
            cloned_affinities[i] = affinities[(positions_antibodies_selected[i])]
            print("Conservamos el clon {} original".format(cloned_antibodies[i]))

    # second part of the function: select R% of the best (efficacy) cloned antibodies
    amount_selected_antibodies = round(len(cloned_efficacies)*R)
    print("{} antibodies were selected".format(amount_selected_antibodies))
    positions = [i #positions of best R% of selected antibodies
        for x, i
        in heapq.nlargest(
            amount_selected_antibodies,
            ((x, i) for i, x in enumerate(cloned_efficacies)))]
    
    for i in range(0,amount_selected_antibodies):
        antibodies[(positions_antibodies_selected[(positions[i])])] = cloned_antibodies[(positions[i])]
        efficacies[(positions_antibodies_selected[(positions[i])])] = cloned_efficacies[(positions[i])]
        affinities[(positions_antibodies_selected[(positions[i])])] = cloned_affinities[(positions[i])]
    return antibodies, efficacies, affinities

In [216]:
antibodies, efficacies, affinities = select_best_cloned_antibodies(antibodies = antibodies,
                                    cloned_antibodies = cloned_antibodies,
                                    efficacies = efficacies, 
                                    cloned_efficacies = cloned_efficacies,
                                    affinities = affinities,
                                    cloned_affinities = cloned_affinities,
                                    R = 0.5,
                                    positions_antibodies_selected = positions_antibodies_selected)
print(efficacies)
print(affinities)
antibodies


0 antibodies were selected
[0.19047619047619044, 0.27272727272727276, 0.16666666666666663]
[0.0, 0.27272727272727276, 0.0]


array([[ 5,  0,  1,  3,  7,  0, 11,  4,  0, 10,  9,  0,  8,  6,  2, 12],
       [ 0, 10, 12,  5,  0,  3,  9,  1,  0, 11,  4,  0,  8,  6,  2,  7],
       [ 7,  5, 10,  0, 12,  0,  0,  8,  9,  6, 11,  4,  0,  1,  3,  2]])

# Remove worst members of the antibodies pool (RECEPTOR EDITING)

"After mutation processes, the
antibodies that have worse efficacy values are erased (worst
%B of the whole population)"

In [217]:
def receptor_editing(antibodies_pool,efficacies,affinities, B):
    amount_antibodies = round(len(efficacies)*B)
    print("\nSe han borrado {} anticuerpos".format(amount_antibodies))
    positions = [i
        for x, i
        in heapq.nsmallest(
            amount_antibodies,
            ((x, i) for i, x in enumerate(efficacies)))]
    # print(positions)
    antibodies_pool = np.delete(antibodies_pool, positions, axis=0)
    efficacies = np.delete(efficacies, positions, axis=0)
    affinities = np.delete(affinities, positions, axis=0)
    return antibodies_pool, efficacies, affinities, amount_antibodies

In [218]:
B = 0.5
antibodies, efficacies, affinities, amount_antibodies_erased = receptor_editing(antibodies_pool = antibodies, 
                                                                                efficacies = efficacies, 
                                                                                affinities=affinities, 
                                                                                B = B)
antibodies


Se han borrado 2 anticuerpos


array([[ 0, 10, 12,  5,  0,  3,  9,  1,  0, 11,  4,  0,  8,  6,  2,  7]])

# New Random antibodies into the population

"Then, same percent of new
antibodies are randomly generated"

La pregunta es la misma cantidad de anticuerpos que hemos borrado en el apartado anterior o el mismo porcentaje (B) sobre la nueva población que se ha visto disminuida por el receptor editing.

In [219]:
number_new_random_antibodies = amount_antibodies_erased

new_random_antibodies = generation_initial_population(p = p, 
                                                    m = m, 
                                                    population_size = number_new_random_antibodies)
print(new_random_antibodies)

antibodies = np.concatenate((antibodies, new_random_antibodies), axis = 0)
antibodies

[[ 0  4  1  0 11  7  3  6  8  5  0  2  9 12 10  0]
 [12  7  3  8  1 10  0  2  0  9  6  0 11  4  5  0]]


array([[ 0, 10, 12,  5,  0,  3,  9,  1,  0, 11,  4,  0,  8,  6,  2,  7],
       [ 0,  4,  1,  0, 11,  7,  3,  6,  8,  5,  0,  2,  9, 12, 10,  0],
       [12,  7,  3,  8,  1, 10,  0,  2,  0,  9,  6,  0, 11,  4,  5,  0]])

# Stopping criteria